In [5]:
import pandas as pd
file_path = 'climate_final_with_ff5_and_excess_return.xlsx'
df = pd.read_excel(file_path, parse_dates=['date'])

df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df['year'] = df['date'].dt.year

# Function to take TOP10% and BOT10% per year on any column
def get_top_bottom_by_year(df, col, q=0.1):
    groups = {}
    for year in sorted(df['year'].dropna().unique()):
        sub = df[df['year'] == year]
        hi = sub[col].quantile(1 - q)
        lo = sub[col].quantile(q)
        suffix = str(int(year))[-2:]
        groups[f'TOP{suffix}'] = sub[sub[col] >= hi]
        groups[f'BOT{suffix}'] = sub[sub[col] <= lo]
    return groups

# Function to run FF-5
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

def run_ff5(groups, ret_col='excess_RET'):
    models = {}
    for name, g in groups.items():
        X = sm.add_constant(g[['Mkt-RF','SMB','HML','RMW','CMA']])
        y = g[ret_col]
        models[name] = sm.OLS(y, X, missing='drop').fit()
    info = {
        'R-squared': lambda m: f"{m.rsquared:.4f}",
        'N':         lambda m: f"{int(m.nobs)}"
    }
    table = summary_col(
        list(models.values()),
        stars=True,
        float_format='%0.4f',
        model_names=list(models.keys()),
        info_dict=info,
        regressor_order=['const','Mkt-RF','SMB','HML','RMW','CMA']
    )
    return table

# Build groups
exp_groups  = get_top_bottom_by_year(df, 'Overall_ESG_Exposure')
sent_groups = get_top_bottom_by_year(df, 'Overall_ESG_Sentiment')

print("=== FF5: ESG Exposure TOP/BOT by Year ===")
print(run_ff5(exp_groups))

print("\n=== FF5: ESG Sentiment TOP/BOT by Year ===")
print(run_ff5(sent_groups))

=== FF5: ESG Exposure TOP/BOT by Year ===

                TOP19    BOT19     TOP20     BOT20     TOP21     BOT21     TOP22     BOT22     TOP23     BOT23     TOP24    BOT24  
-----------------------------------------------------------------------------------------------------------------------------------
const          -0.0098  0.0127   0.0077    -0.0277   -0.0106   0.0033    0.0144*   0.0011    -0.0172*  0.0093    0.0128    -0.0252*
               (0.0096) (0.0158) (0.0159)  (0.0196)  (0.0099)  (0.0121)  (0.0076)  (0.0113)  (0.0097)  (0.0134)  (0.0094)  (0.0132)
Mkt-RF         0.7475** 0.7908*  0.9488*** 0.6603*** 1.1128*** 1.0603*** 0.8174*** 0.9420*** 0.9807*** 0.9857*** 1.2397*** 1.0890**
               (0.2824) (0.4630) (0.1903)  (0.2337)  (0.2782)  (0.3388)  (0.1056)  (0.1559)  (0.2034)  (0.2805)  (0.3058)  (0.4314)
SMB            -0.7981  1.4799   -0.8817** 0.8592    -0.9717** 0.0365    -0.5441   -0.4422   -0.2342   -0.0661   1.2611*** 0.4676  
               (0.6713) (1.1004) 

In [7]:
# --- Cell: Descriptive statistics on ESG Exposure & Sentiment ---

# assume df is already loaded, date parsed, and df['year'] exists
# file loading for reference:
# df = pd.read_excel('climate_final_with_ff5_and_excess_return.xlsx')
# df['date'] = pd.to_datetime(df['date'], dayfirst=True)
# df['year'] = df['date'].dt.year

esg_cols = ['Overall_ESG_Exposure', 'Overall_ESG_Sentiment']

# 1. Overall descriptive statistics
desc_overall = df[esg_cols].describe().T
desc_overall.columns = ['Count', 'Mean', 'Std', 'Min', '25%', '50%', '75%', 'Max']
print(">>> Overall Descriptive Statistics\n")
print(desc_overall)

# 2. Yearly descriptive statistics
# groupby.describe() produces a MultiIndex; we'll unstack it for readability
desc_by_year = df.groupby('year')[esg_cols].describe().unstack(level=0)

# flatten the column index
desc_by_year.columns = [
    f"{stat}_{year}" for (col, year), stat in desc_by_year.columns.to_flat_index()
]
print("\n>>> Yearly Descriptive Statistics\n")
print(desc_by_year)

# 3. (Optional) Export to Excel
# with pd.ExcelWriter('esg_descriptives.xlsx') as writer:
#     desc_overall.to_excel(writer, sheet_name='Overall')
#     desc_by_year.to_excel(writer, sheet_name='By_Year')

>>> Overall Descriptive Statistics

                        Count      Mean       Std       Min       25%  \
Overall_ESG_Exposure   4842.0  0.095394  0.120853  0.000000  0.016250   
Overall_ESG_Sentiment  4842.0  0.250600  0.149102 -0.099747  0.156646   

                            50%       75%       Max  
Overall_ESG_Exposure   0.038684  0.113445  0.750000  
Overall_ESG_Sentiment  0.251353  0.344945  0.702981  


AttributeError: 'Series' object has no attribute 'columns'

In [9]:
file_path = 'climate_final_with_ff5_and_excess_return.xlsx'
df = pd.read_excel(file_path, parse_dates=['date'])

df['date'] = pd.to_datetime(df['date'], dayfirst=True, errors='coerce')
df['year'] = df['date'].dt.year

# Function to take TOP10% and BOT10% per year on any column
def get_top_bottom_by_year(df, col, q=0.1):
    groups = {}
    for year in sorted(df['year'].dropna().unique()):
        sub = df[df['year'] == year]
        hi = sub[col].quantile(1 - q)
        lo = sub[col].quantile(q)
        suffix = str(int(year))[-2:]
        groups[f'TOP{suffix}'] = sub[sub[col] >= hi]
        groups[f'BOT{suffix}'] = sub[sub[col] <= lo]
    return groups

# Function to run FF-3
import statsmodels.api as sm
from statsmodels.iolib.summary2 import summary_col

def run_ff3(groups, ret_col='excess_RET'):
    models = {}
    for name, g in groups.items():
        X = sm.add_constant(g[['Mkt-RF', 'SMB', 'HML']])  # Only 3 factors
        y = g[ret_col]
        models[name] = sm.OLS(y, X, missing='drop').fit()
    info = {
        'R-squared': lambda m: f"{m.rsquared:.4f}",
        'N':         lambda m: f"{int(m.nobs)}"
    }
    table = summary_col(
        list(models.values()),
        stars=True,
        float_format='%0.4f',
        model_names=list(models.keys()),
        info_dict=info,
        regressor_order=['const', 'Mkt-RF', 'SMB', 'HML']
    )
    return table

# Build groups
exp_groups  = get_top_bottom_by_year(df, 'Overall_ESG_Exposure')
sent_groups = get_top_bottom_by_year(df, 'Overall_ESG_Sentiment')

print("=== FF3: ESG Exposure TOP/BOT by Year ===")
print(run_ff3(exp_groups))

print("\n=== FF3: ESG Sentiment TOP/BOT by Year ===")
print(run_ff3(sent_groups))

=== FF3: ESG Exposure TOP/BOT by Year ===

                TOP19     BOT19     TOP20     BOT20     TOP21      BOT21     TOP22     BOT22     TOP23      BOT23    TOP24    BOT24  
-------------------------------------------------------------------------------------------------------------------------------------
const          -0.0004  0.0118    0.0171    -0.0239   -0.0021    0.0011    0.0185**  0.0053    -0.0173**  0.0086    0.0068   -0.0209*
               (0.0081) (0.0128)  (0.0154)  (0.0185)  (0.0092)   (0.0109)  (0.0076)  (0.0106)  (0.0070)   (0.0094)  (0.0090) (0.0119)
Mkt-RF         0.4571** 0.8605*** 0.7333*** 0.6429*** 0.8280***  1.2375*** 0.7687*** 0.9937*** 0.8840***  0.9790*** 0.6475** 0.8970**
               (0.1831) (0.2913)  (0.1405)  (0.1691)  (0.2615)   (0.3094)  (0.1048)  (0.1465)  (0.1747)   (0.2342)  (0.2821) (0.3717)
SMB            -0.8384* 1.2061    -0.5112   0.9079*   -0.7250*** -0.4112   -0.7264** -0.6674   -0.6416*** -0.1395   0.1540   0.1600  
               (0.4